## FEM for the clover 

In [51]:
from ngsolve import *
from netgen.read_gmsh import ReadGmsh
from netgen.csg import *
import numpy as np
import meshio
import scipy.special as sc
import scipy.sparse as sp
import scipy.sparse.linalg as la
import matplotlib.pyplot as plt

In [54]:
#create mesh
mesh = Mesh(ReadGmsh("clover_mesh.msh"))
mesh2=meshio.read("clover_mesh.msh")

In [49]:
List_of_mats = ['metal', 'vacuum']
hankel = lambda n, k : sc.hankel1(n,k)
dhankel = lambda n, k : sc.h1vp(n,k,1)

In [50]:
def ngvec_to_vec(vector):
    returnvec=np.zeros(vector.size,dtype='complex')
    for i in range(0,vector.size):
        returnvec[i]=vector.data[i]
    return returnvec

In [42]:
def vec_to_ngvec(vec,ngvec):
    for i in range(0,vec.size):
        ngvec.data[i]=vec[i]

In [6]:
#convert ngsolve solution to vector
def ConvertSolutiononMesh(mesh,meshpts,gfu):
    soln = np.zeros(meshpts.shape[0],dtype='complex')
    for i in range(0,meshpts.shape[0]):
        soln[i] = gfu(mesh(meshpts[i,0],meshpts[i,1],meshpts[i,2]))
    return soln

In [20]:
def FEM(epsm,order,k,DtN_tol):
    R=3
    #define epsilon parameter
    epsv=1
    eps_val={'metal' : epsm, 'vacuum' : epsv}
    eps=CoefficientFunction([eps_val[mat] for mat in List_of_mats])
    epsinv_val = {'metal' : 1/epsm, 'vacuum' : 1/epsv}
    epsinv=CoefficientFunction([epsinv_val[mat] for mat in List_of_mats])
    
    #create finite element space
    fes = H1(mesh, order=order, complex=True)
   
    #define incident field
    Inc = CoefficientFunction(exp(1j*k*y))
    diff_Inc_x = Inc.Diff(x)
    diff_Inc_y = Inc.Diff(y)
    IncNormDrv = CoefficientFunction((1/sqrt(x*x+y*y))*(x*diff_Inc_x+y*diff_Inc_y))
    
    #prepare and populate linear/bilinear forms
    u, v = fes.TnT()
    a = BilinearForm(fes)
    f = LinearForm(fes)
    a += (-1*epsinv*(grad(u)*grad(v)))*dx
    a += (k**2*u*v)*dx
    ginc = (1/epsv)*(-1*IncNormDrv)
    f += (ginc*v)*ds('Boundary')
    
    #create DtN
    dtn1_array= np.zeros([2*DtN_tol+1],dtype='object')
    dtn_mat=sp.csr_matrix(np.zeros([fes.ndof,fes.ndof],dtype="complex"))
    ulminc_mat=np.zeros([2*DtN_tol+1],dtype="complex")
    ulm_mat=np.zeros([2*DtN_tol+1],dtype="complex")
    const_mat=np.zeros([2*DtN_tol+1],dtype='complex')     
    rhs_array=np.zeros([fes.ndof],dtype='complex')
    #print("Creating dtn matrices")
    
    for l in range(-DtN_tol,DtN_tol+1):
        ulminc_mat[l+DtN_tol] = Integrate(Inc*Conj(exp(1j*l*atan2(y,x))),mesh,definedon=mesh.Boundaries("Boundary"))
        
        #create matching constant for dtn expansion
        const_mat[l+DtN_tol]=k*(dhankel(np.abs(l),k*R)/hankel(np.abs(l),k*R))
                    
        #create vector associated with v - dtn applied to all phi
        
        dtn1_array[l+DtN_tol]= LinearForm(fes)
        dtn1_array[l+DtN_tol]+=(v*Conj(exp(1j*l*atan2(y,x))))*ds('Boundary')
        dtn1_array[l+DtN_tol].Assemble()
        #Right hand side - dtn applied to incident field
        #constant*u^inc_lm*v_lm
        rhs_array += const_mat[l+DtN_tol]*ulminc_mat[l+DtN_tol]*np.conjugate(ngvec_to_vec(dtn1_array[l+DtN_tol].vec))
                    
        #build vectors
                    
        dtn_vec1=np.conjugate(ngvec_to_vec(dtn1_array[l+DtN_tol].vec))
        dtn_vec2=ngvec_to_vec(dtn1_array[l+DtN_tol].vec)
                    
        #rows of the column vector correspond to row values of the matrix - we want the i-th 
        #row to be associated with a fixed ϕ over which we compose u as a basis of all ϕ
        dtn_mat += const_mat[l+DtN_tol]*(sp.csr_matrix(dtn_vec1).T*sp.csr_matrix(dtn_vec2))
        #print("Updated matrix with dtn l=" +str(l)+" m=" + str(m))
    a.Assemble()
    f.Assemble()
    rhs_vec=ngvec_to_vec(f.vec)+2*np.pi*R*rhs_array
    Amat=a.mat
    rows2,cols2,vals2 = Amat.COO()
    Asparse=sp.csr_matrix((vals2,(rows2,cols2)))
    LHS=Asparse+2*np.pi*R*dtn_mat
    return dtn_vec1
    vec_sols=la.spsolve(LHS,rhs_vec)
    gfu = GridFunction(fes)
    vec_to_ngvec(vec_sols,gfu.vec.data)
    FEMSoln = ConvertSolutiononMesh(mesh,mesh2.points,gfu)   

In [21]:
save=FEM(-1.1,2,5,15)

In [59]:
fes = H1(mesh, order=2, complex=True)
#prepare and populate linear/bilinear forms
u, v = fes.TnT()
a = BilinearForm(fes)
f = LinearForm(fes)

In [60]:
f+= v*1*ds('outer')

In [61]:
ngvec_to_vec(f.vec)

array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])

In [ ]:
mesh.GetB